# Train transformer

In [7]:
import torch
from torch import nn
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import MultiStepLR
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoTokenizer
from utils.data_utils import AG_NEWS_DATASET
from utils.constants import *
from model.transformer import Transformer as Transformer_origin
from utils.training import Learner

from quantization.binarize import binarize, binarize_origin
from quantization.transformer_raw import Transformer

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# load dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
train_dl, test_dl = AG_NEWS_DATASET(tokenizer, batch_size = BATCH_SIZE).load_data()

# create model
model = Transformer(d_model=BASELINE_MODEL_DIM,
                    d_ff=BASELINE_FFN_DIM,
                    d_hidden=BASELINE_HIDDEN_DIM,
                    h=BASELINE_MODEL_NUMBER_OF_HEADS,
                    n_layers=BASELINE_MODEL_NUMBER_OF_LAYERS,
                    n_class=4,
                    vocab=tokenizer.vocab_size
                   )

binarize(model, 'FFN_ONLY')
print(model)

# loss func
loss_fn = nn.CrossEntropyLoss()

# baseline training config -> do not change!
optim = Adam(model.parameters(), lr= 1e-4)
scheduler = MultiStepLR(optim, milestones=[10,15], gamma=0.1)

train_config ={'model': model,
               'loss_fn': loss_fn,
               'optim': optim,
               'scheduler': scheduler,
               'datasets': [train_dl, test_dl],
               'epochs': 10,
               'batch_size': BATCH_SIZE
               }

train_config['exp_name'] = 'transformer_binarization_FFN_ONLY'

# training
learner_ag_news = Learner(train_config)

Transformer(
  (input_embeddings): Embeddings(
    (token_embedding): Embedding(30522, 512)
    (pos_embedding): Embedding(512, 512)
  )
  (input_encodings): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sublayer_attention): ModuleList(
    (0): sublayerConnectionAttention(
      (multiheads): MultiheadAttention(
        (heads): ModuleList(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): Linear(in_features=512, out_features=512, bias=True)
          (2): Linear(in_features=512, out_features=512, bias=True)
        )
        (output): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layernorm): LayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): sublayerConnectionAttention(
      (multiheads): MultiheadAttention(
        (heads): ModuleList(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): Linear(in_feat

In [15]:
learner_ag_news.train()

  0%|                                                    | 0/10 [00:00<?, ?it/s]

current lr 1.00000e-04
Epoch: [0][0/3750]	Loss 790.5084	Prec@1 3.125
Epoch: [0][100/3750]	Loss 794.2428	Prec@1 31.900
Epoch: [0][200/3750]	Loss 483.2325	Prec@1 28.638
Epoch: [0][300/3750]	Loss 364.5698	Prec@1 28.644
Epoch: [0][400/3750]	Loss 302.0592	Prec@1 28.499
Epoch: [0][500/3750]	Loss 262.1268	Prec@1 28.493
Epoch: [0][600/3750]	Loss 234.1791	Prec@1 28.291
Epoch: [0][700/3750]	Loss 211.3285	Prec@1 28.196
Epoch: [0][800/3750]	Loss 193.3300	Prec@1 28.223
Epoch: [0][900/3750]	Loss 178.5055	Prec@1 28.014
Epoch: [0][1000/3750]	Loss 165.4602	Prec@1 28.131
Epoch: [0][1100/3750]	Loss 154.2756	Prec@1 28.247
Epoch: [0][1200/3750]	Loss 144.7231	Prec@1 28.193
Epoch: [0][1300/3750]	Loss 136.2696	Prec@1 28.192
Epoch: [0][1400/3750]	Loss 128.7231	Prec@1 28.129
Epoch: [0][1500/3750]	Loss 121.8323	Prec@1 28.115
Epoch: [0][1600/3750]	Loss 115.6597	Prec@1 28.111
Epoch: [0][1700/3750]	Loss 110.0748	Prec@1 28.072
Epoch: [0][1800/3750]	Loss 104.9048	Prec@1 28.212
Epoch: [0][1900/3750]	Loss 100.1867	Prec

 10%|████▍                                       | 1/10 [01:30<13:35, 90.60s/it]

current lr 1.00000e-04
Epoch: [1][0/3750]	Loss 1.7176	Prec@1 9.375
Epoch: [1][100/3750]	Loss 1.8472	Prec@1 27.785
Epoch: [1][200/3750]	Loss 1.6973	Prec@1 28.172
Epoch: [1][300/3750]	Loss 1.6448	Prec@1 28.270
Epoch: [1][400/3750]	Loss 1.6110	Prec@1 28.296
Epoch: [1][500/3750]	Loss 1.5763	Prec@1 29.279
Epoch: [1][600/3750]	Loss 1.5645	Prec@1 29.134
Epoch: [1][700/3750]	Loss 1.5599	Prec@1 28.950
Epoch: [1][800/3750]	Loss 1.5456	Prec@1 28.983
Epoch: [1][900/3750]	Loss 1.5380	Prec@1 29.030
Epoch: [1][1000/3750]	Loss 1.5301	Prec@1 28.993
Epoch: [1][1100/3750]	Loss 1.5193	Prec@1 29.314
Epoch: [1][1200/3750]	Loss 1.5140	Prec@1 29.366
Epoch: [1][1300/3750]	Loss 1.5064	Prec@1 29.624
Epoch: [1][1400/3750]	Loss 1.5021	Prec@1 29.550
Epoch: [1][1500/3750]	Loss 1.4976	Prec@1 29.620
Epoch: [1][1600/3750]	Loss 1.4932	Prec@1 29.614
Epoch: [1][1700/3750]	Loss 1.4908	Prec@1 29.573
Epoch: [1][1800/3750]	Loss 1.4856	Prec@1 29.782
Epoch: [1][1900/3750]	Loss 1.4835	Prec@1 29.748
Epoch: [1][2000/3750]	Loss 1.4

 20%|████████▊                                   | 2/10 [03:02<12:09, 91.23s/it]

...

 90%|███████████████████████████████████████▌    | 9/10 [13:38<01:30, 90.55s/it]

Epoch[8] *Validation*: Prec@1 90.066
current lr 1.00000e-04
Epoch: [9][0/3750]	Loss 0.0880	Prec@1 93.750
Epoch: [9][100/3750]	Loss 0.1418	Prec@1 96.040
Epoch: [9][200/3750]	Loss 0.1071	Prec@1 96.782
Epoch: [9][300/3750]	Loss 0.0917	Prec@1 97.124
Epoch: [9][400/3750]	Loss 0.0799	Prec@1 97.452
Epoch: [9][500/3750]	Loss 0.0733	Prec@1 97.686
Epoch: [9][600/3750]	Loss 0.0697	Prec@1 97.769
Epoch: [9][700/3750]	Loss 0.0641	Prec@1 97.958
Epoch: [9][800/3750]	Loss 0.0603	Prec@1 98.061
Epoch: [9][900/3750]	Loss 0.0582	Prec@1 98.124
Epoch: [9][1000/3750]	Loss 0.0565	Prec@1 98.177
Epoch: [9][1100/3750]	Loss 0.0541	Prec@1 98.232
Epoch: [9][1200/3750]	Loss 0.0527	Prec@1 98.280
Epoch: [9][1300/3750]	Loss 0.0514	Prec@1 98.321
Epoch: [9][1400/3750]	Loss 0.0509	Prec@1 98.323
Epoch: [9][1500/3750]	Loss 0.0516	Prec@1 98.299
Epoch: [9][1600/3750]	Loss 0.0508	Prec@1 98.321
Epoch: [9][1700/3750]	Loss 0.0502	Prec@1 98.341
Epoch: [9][1800/3750]	Loss 0.0493	Prec@1 98.362
Epoch: [9][1900/3750]	Loss 0.0494	Prec@1

100%|███████████████████████████████████████████| 10/10 [15:10<00:00, 91.01s/it]

Epoch[9] *Validation*: Prec@1 90.329


[26.63157894736842,
 51.55263157894737,
 89.6842105263158,
 89.92105263157895,
 90.01315789473684,
 90.26315789473684,
 90.25,
 90.52631578947368,
 90.0657894736842,
 90.32894736842105]

# Memory compute

In [67]:
from utils.utils import count_memory_size

In [68]:
count_memory_size(model)

82497552

In [69]:
total = count_memory_size(model) - count_memory_size(model.input_embeddings)
for name, layer in model.named_children():
    print(f'{name}: {count_memory_size(layer)/total}')

input_embeddings: 3.355749760294658
input_encodings: 0.0
sublayer_attention: 0.44420377699589014
sublayer_ffn: 0.4439875142028055
classifier: 0.11180870880130434
